<a href="https://colab.research.google.com/github/elikapati/DeepLearning/blob/master/ClassifyAnimal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Classification of Cat and Dog using Keras
# Dataset - https://www.kaggle.com/c/dogs-vs-cats/data

In [0]:
import numpy as np
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Activation
from keras.layers.core import Dense, Flatten
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import *
from matplotlib import pyplot as plt
from sklearn.metrics import confusion_matrix
import itertools
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
train_path = 'drive/My Drive/Colab Notebooks/data/cats-and-dogs/train'
valid_path = 'drive/My Drive/Colab Notebooks/data/cats-and-dogs/valid'
test_path = 'drive/My Drive/Colab Notebooks/data/cats-and-dogs/test'

In [0]:
print(train_path)
train_batches = ImageDataGenerator().flow_from_directory(train_path, target_size=(224, 224), classes=['dog', 'cat'], batch_size=10)
valid_batches = ImageDataGenerator().flow_from_directory(valid_path, target_size=(224, 224), classes=['dog', 'cat'], batch_size=4)
test_batches = ImageDataGenerator().flow_from_directory(test_path, target_size=(224, 224), classes=['dog', 'cat'], batch_size=10)

In [0]:
# plot images with labels within jupyter notebook
def plots(ims, figsize=(12, 6), rows=1, interp=False, titles=None):
    if type(ims[0]) is np.ndarray:
        ims = np.array(ims).astype(np.uint8)
        if (ims.shape[-1] != 3):
            ims = ims.transpose((0, 2, 3, 1))
    f = plt.figure(figsize=figsize)
    cols = len(ims)//rows if len(ims) % 2 == 0 else len(ims)//rows + 1
    for i in range(len(ims)):
        sp = f.add_subplot(rows, cols, i+1)
        sp.axis('off')
        if titles is not None:
            sp.set_title(titles[i], fontsize=16)
        plt.imshow(ims[i], interpolation=None if interp else 'none')

In [0]:
imgs, labels = next(train_batches)

In [0]:
plots(imgs, titles=labels)

#### Build and Train CNN

In [0]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    Flatten(),
    Dense(2, activation='softmax')
])

In [0]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=4, epochs=5, verbose=2)

#### Predict

In [0]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [0]:
test_labels = test_labels[:, 0]
test_labels

In [0]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)
predictions = predictions.astype('int')
predictions

In [0]:
cm = confusion_matrix(test_labels, predictions[:, 0])

In [0]:
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    """This function prints and plots the confusion matrix.
       Normalization can be applied by setting 'normalize=True'.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized Confusion Matrix")
    else:
        print('Confusion matrix, without normalization')
        
    print(cm)
    
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment = 'center', color='white' if cm[i, j] > thresh else 'black')
    
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [0]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [0]:
vgg16_model = keras.applications.vgg16.VGG16()

In [0]:
vgg16_model.summary()

In [0]:
type(vgg16_model)

In [0]:
# Convert to Sequential model
model = Sequential()
for layer in vgg16_model.layers[:-1]:
    model.add(layer)

In [0]:
# Verify if the last layer "predictions" is removed from the model 
# Note: model.pop() is equivalent but failed so we used above code to add all layers but the last one to the model
model.summary()

In [0]:
# We want use the model as it is; we don't want to train the model with current training data 
# We don't want to change its weights
for layer in model.layers:
    layer.trainable = False

In [0]:
# Add our layer to output 2 values Dog or Cat
model.add(Dense(2, activation='softmax'))

In [0]:
model.summary()

#### Train the fine-tuned VGG16 model

In [0]:
model.compile(Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
model.fit_generator(train_batches, steps_per_epoch=4, validation_data=valid_batches, validation_steps=4, epochs=5, verbose=2)

#### Predict using fine-tuned VGG16 model

In [0]:
test_imgs, test_labels = next(test_batches)
plots(test_imgs, titles=test_labels)

In [0]:
test_labels = test_labels[:, 0]
test_labels

In [0]:
predictions = model.predict_generator(test_batches, steps=1, verbose=0)

In [0]:
cm = confusion_matrix(test_labels, np.round(predictions[:, 0]))

In [0]:
cm_plot_labels = ['cat', 'dog']
plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')